In [1]:
# -- Load module for singularity -- 

!module load singularity
!module load freesurfer
!module load fsl

import os 
import subprocess
import pandas as pd

from IPython.display import display, Markdown, Latex

# -- Init Variables -- 
scripts_folder='/home/traaffneu/margal/awake_code/awake/scripts/'
metadata_path ='/home/traaffneu/margal/awake_code/awake/scripts/tables/metadata_rabies.tsv'

df = pd.read_csv(metadata_path, sep='\t')
df = df.loc[(df['exclude'] != 'yes')]


In [2]:
# --- Function to launch RABIES as a job ---

def queue_process(specie, template_dir, input_bids_dir, output_dir, TR, template, mask, wm, csf, atlas, smoothing):
    
    singularity_path='/opt/singularity/3.10.3/bin/singularity'

    # -- Create the full command string --
    
    if specie == 'Rat' :
        rabies_cmd = f"{singularity_path} run " \
                f"-B {template_dir}:/template " \
                f"-B {input_bids_dir}:/rabies_in:ro " \
                f"-B {output_dir}:/rabies_out " \
                "/opt/rabies/0.5.0/rabies.sif -p MultiProc preprocess /rabies_in /rabies_out " \
                    f"--TR {TR} " \
                    f"--anat_template {template} " \
                    f"--brain_mask {mask} " \
                    f"--WM_mask {wm} " \
                    f"--CSF_mask {csf} " \
                    f"--vascular_mask {csf} " \
                    f"--labels {atlas} " \
                    "--commonspace_reg='masking'='false','brain_extraction'='false','template_registration'='SyN','fast_commonspace'='true' " \
                    "--commonspace_resampling '0.3x0.3x0.3' " \
    
    else :
        rabies_cmd = f"{singularity_path} run " \
                    f"-B {input_bids_dir}:/rabies_in:ro " \
                    f"-B {output_dir}:/rabies_out " \
                    "/opt/rabies/0.5.0/rabies.sif -p MultiProc preprocess /rabies_in /rabies_out " \
                        f"--TR {TR} " \
                        "--commonspace_reg='masking'='false','brain_extraction'='false','template_registration'='SyN','fast_commonspace'='true' " \
                        "--commonspace_resampling '0.3x0.3x0.3' " \
               
    confound_cmd = "/opt/singularity/3.10.3/bin/singularity run " \
                f"-B {input_bids_dir}:/rabies_in:ro " \
                f"-B {output_dir}/preprocess_outputs:/preprocess_outputs " \
                "/opt/rabies/0.5.0/rabies.sif -p MultiProc confound_correction /preprocess_outputs/ /confound_correction_outputs/ " \
                    f"--smoothing_filter {smoothing} " \
                    "--highpass 0.01 " \
                    "--lowpass 0.1 " \
                                   
                #correction_arg, # Iterativement, if precedent arg fail
                    # 1 "--anat_autobox\ --bold_autobox"
                    # 2 "--anat_autobox --bold_autobox --bold_inho_cor=method=N4_reg,otsu_thresh=2,multiotsu=false --anat_inho_cor=method=N4_reg,otsu_thresh=2,multiotsu=false"
                    # 3 "--anat_autobox --bold_autobox --bold_inho_cor=method=N4_reg,otsu_thresh=2,multiotsu=false --anat_inho_cor=method=N4_reg,otsu_thresh=2,multiotsu=false --bold2anat_coreg=masking=false,brain_extraction=false,registration=Rigid"
                
    # Submit the job
    combined_cmd = f'{rabies_cmd} && {confound_cmd}'
    cmd = f'echo "{combined_cmd}" | qsub -N Awake:{specie} -l nodes=1:ppn=1,mem=64gb,walltime=20:00:00'
    subprocess.call(cmd, shell=True)
    
    display(Markdown('**Job running**'))


In [5]:
# --- Run RABIES ---
 
for index in range(0, len(df)):
    
    # -- Define variables --
    subj_num=str(df.iloc[index]['rodent.sub'])[:-2]               #remove the 2 last characters with [:-2]
    ses_num=str(df.iloc[index]['rodent.ses'])[:-2] 
    specie = str(df.iloc[index]['rodent.spiecies'])   
    template_dir='/groupshare/traaffneu/preclinimg/templates/SIGMA_Wistar_Rat_Brain_TemplatesAndAtlases_Version1.1'              
    bids_folder = '02_rats'    
    smoothing = 0.5
    
    if specie == 'Mouse' :
        bids_folder = '01_mice'
        template_folder=''
        template=''
        mask=''
        wm=''
        csf=''
        atlas=''
        roi=''

        smoothing = 0.3

    else : #'Rat'
        bids_folder = '02_rats'    
        template_folder='/groupshare/traaffneu/preclinimg/templates/SIGMA_Wistar_Rat_Brain_TemplatesAndAtlases_Version1.1'
        template='/template/SIGMA_Rat_Anatomical_Imaging/SIGMA_Rat_Anatomical_InVivo_Template/SIGMA_InVivo_Brain_Template.nii'
        mask='/template/SIGMA_Rat_Anatomical_Imaging/SIGMA_Rat_Anatomical_InVivo_Template/SIGMA_InVivo_Brain_Mask.nii'
        wm='/template/SIGMA_Rat_Anatomical_Imaging/SIGMA_Rat_Anatomical_InVivo_Template/SIGMA_InVivo_WM_bin.nii.gz'
        csf='/template/SIGMA_Rat_Anatomical_Imaging/SIGMA_Rat_Anatomical_InVivo_Template/SIGMA_InVivo_CSF_bin.nii.gz'
        atlas='/template/SIGMA_Rat_Brain_Atlases/SIGMA_Anatomical_Atlas/SIGMA_Anatomical_Brain_Atlas_rs.nii'
        roi='/groupshare/traaffneu/preclinimg/template/roi/'
        smoothing = 0.5

    TR=str(df.iloc[index]['func.TR'])[:-2]
    correction_arg = str(df.iloc[index]['rabies_cor'])
    rabies_preprocess = str(df.iloc[index]['rabies_prepro'])       

    # -- Input and output directories -- 
    orig_bids_dir=f'/project/4180000.36/AwakeRodent/bids/{bids_folder}/'
    input_bids_dir=f'/project/4180000.36/AwakeRodent/scratch/bids/{bids_folder}/sub-0{subj_num}'
    output_dir=f'/project/4180000.36/AwakeRodent/scratch/RABIES_preprocess/sub-0{subj_num}_ses-{ses_num}'

    # -- Print scan infos -- 
    display(Markdown(f'Specie: **<span style="color:#FFA07A">{specie}</span>**   \n'
                f'Rodent num: <span style="color:#FF5733">0{subj_num}</span>  \n'     
                f'Preprocess tatus: <span style="color:#FF91A4">{rabies_preprocess}</span>  \n'
                f'Session n°: <span style="color:#45B39D">{ses_num}</span>  \n'
                f'TR: <span style="color:#0099FF">{TR}</span>  \n'))
    
    
    # -- Run RABIES as a job on the HPC --
    if rabies_preprocess == '2' :
        if not os.path.exists(output_dir):os.makedirs(output_dir)      # Make directories
        queue_process(specie, template_dir, input_bids_dir, output_dir, TR, template, mask, wm, csf, atlas, smoothing)
        print("RABIES: corrective params.")
        print('-----------')       

    elif rabies_preprocess == '0' :
        if not os.path.exists(output_dir):os.makedirs(output_dir)      # Make directories
        queue_process(specie, template_dir, input_bids_dir, output_dir, TR, template, mask, wm, csf, atlas, smoothing)
        print("RABIES: default params.") 
        print('-----------')   
            
    else :                                                             # if rabies_preprocess == 1 :
        print("RABIES: already prepro.")
        print('-----------')       
        continue # go to next iteration


Specie: **<span style="color:#FFA07A">Mouse</span>**   
Rodent num: <span style="color:#FF5733">0100100</span>  
Preprocess tatus: <span style="color:#FF91A4">0</span>  
Session n°: <span style="color:#45B39D">1</span>  
TR: <span style="color:#0099FF">1000</span>  


50407281.dccn-l029.dccn.nl


**Job running**

RABIES: default params.
-----------


Specie: **<span style="color:#FFA07A">Rat</span>**   
Rodent num: <span style="color:#FF5733">0200100</span>  
Preprocess tatus: <span style="color:#FF91A4">0</span>  
Session n°: <span style="color:#45B39D">1</span>  
TR: <span style="color:#0099FF">1000</span>  


50407282.dccn-l029.dccn.nl


**Job running**

RABIES: default params.
-----------
